# Extracting the Input Instance from the Dicom Files

In [48]:
import os
import pydicom
from shutil import copyfile

# Specify the path to the CT DICOM folder
folder_path = 'D:\\Jupyter Notebooks\\Freelancing\\Monai Dicom\\01 Carotid\\IMAGES'

# Get a list of all DICOM files in the folder
dicom_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]

# Create a dictionary to store the series files
series_files = {}

# Loop through each DICOM file and group them by series
for file_path in dicom_files:
    ds = pydicom.dcmread(file_path)
    series_instance_uid = ds.SeriesInstanceUID
    
    if series_instance_uid not in series_files:
        series_files[series_instance_uid] = []
    
    series_files[series_instance_uid].append(file_path)

# Loop through each series and extract the files
for series_uid, files in series_files.items():
    # Create a new folder for each series
    series_folder_path = os.path.join(folder_path, f'Series_{series_uid}')
    os.makedirs(series_folder_path, exist_ok=True)
    
    # Copy the DICOM files to the series folder
    for file_path in files:
        file_name = os.path.basename(file_path)
        destination_path = os.path.join(series_folder_path, file_name)
        copyfile(file_path, destination_path)


# Image Pre-Processing

In [45]:
import cv2
import pydicom
import numpy as np

# Read the DICOM image
ds = pydicom.dcmread('D:\\Jupyter Notebooks\\Freelancing\\Monai Dicom\\01 Carotid\\Input Dicom\\IM00227')
image = ds.pixel_array

# Check if the image is single-channel
if len(image.shape) == 2:
    # Convert single-channel image to 3-channel (BGR)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# Apply image processing techniques
# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply histogram equalization to enhance contrast
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
equalized = clahe.apply(gray)

# Apply adaptive thresholding to improve image clarity
_, thresholded = cv2.threshold(equalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# convert to uint8
thresholded = np.uint8(thresholded)

# Apply image denoising using Non-Local Means Denoising
denoised = cv2.fastNlMeansDenoising(thresholded, None, 10, 7, 21)

# Apply morphological operations to further enhance the image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
morphology = cv2.morphologyEx(denoised, cv2.MORPH_CLOSE, kernel)

# Display the processed image
cv2.imshow('Processed Image', morphology)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [47]:
dir(ds)

['AccessionNumber',
 'AcquisitionDate',
 'AcquisitionDateTime',
 'AcquisitionNumber',
 'AcquisitionTime',
 'AcquisitionType',
 'AdmittingDiagnosesDescription',
 'BitsAllocated',
 'BitsStored',
 'BodyPartExamined',
 'CTDIvol',
 'Columns',
 'ContentDate',
 'ContentTime',
 'ContrastBolusAgent',
 'ContrastBolusIngredientConcentration',
 'ContrastBolusRoute',
 'ContrastBolusTotalDose',
 'ContrastBolusVolume',
 'ContrastFlowDuration',
 'ContrastFlowRate',
 'ConvolutionKernel',
 'DataCollectionDiameter',
 'DeviceSerialNumber',
 'DistanceSourceToDetector',
 'DistanceSourceToPatient',
 'EstimatedDoseSaving',
 'Exposure',
 'ExposureModulationType',
 'ExposureTime',
 'FilterType',
 'FrameOfReferenceUID',
 'GantryDetectorTilt',
 'HighBit',
 'ImageComments',
 'ImageOrientationPatient',
 'ImagePositionPatient',
 'ImageType',
 'InstanceCreationDate',
 'InstanceCreationTime',
 'InstanceNumber',
 'InstitutionName',
 'IrradiationEventUID',
 'KVP',
 'Manufacturer',
 'ManufacturerModelName',
 'Modality',
